In [ ]:
from model import Model
from util import convert_tokens, get_batch_dataset, get_dataset, get_record_parser
import tensorflow as tf
from config import flags
import numpy as np
import json

In [ ]:
flags.DEFINE_string('f', 'give up already', 'who cares lol')
config = flags.FLAGS

In [ ]:
test_eval_file = 'data/retraining_dev_meta.json'
test_record_file = 'data/retraining_dev.tf'

with open(test_eval_file, "r") as fh:
    eval_file = json.load(fh)

meta = {'total': 1000}

In [ ]:
with open(config.word_emb_file, "r") as fh:
    word_mat = np.array(json.load(fh), dtype=np.float32)
with open(config.char_emb_file, "r") as fh:
    char_mat = np.array(json.load(fh), dtype=np.float32)

In [ ]:
parser = get_record_parser(config)
train_dataset = get_batch_dataset('data/retraining_train.tf', parser, config)
dev_dataset = get_dataset('data/retraining_dev.tf', parser, config)
handle = tf.placeholder(tf.string, shape=[])
iterator = tf.data.Iterator.from_string_handle(
    handle, train_dataset.output_types, train_dataset.output_shapes)
train_iterator = train_dataset.make_one_shot_iterator()
dev_iterator = dev_dataset.make_one_shot_iterator()

In [ ]:
model = Model(config, iterator, word_mat, char_mat)

In [ ]:
sess_config = tf.ConfigProto(allow_soft_placement=True)
sess_config.gpu_options.allow_growth = True

sess = tf.Session(config=sess_config)

In [ ]:
sess.run(tf.global_variables_initializer())
saver = tf.train.Saver(var_list=[v for v in tf.global_variables() if not v.name.startswith('encoding_1/') and not v.name.startswith('badptr') and '/Adam' not in v.name and 'beta' not in v.name])
print('Restoring', tf.train.latest_checkpoint(config.save_dir))
saver.restore(sess, tf.train.latest_checkpoint(config.save_dir))
sess.run(tf.assign(model.is_train, tf.constant(True, dtype=tf.bool)))

In [ ]:
best_acc = 0
saver2 = tf.train.Saver()
train_handle = sess.run(train_iterator.string_handle())
dev_handle = sess.run(dev_iterator.string_handle())

In [ ]:
for _ in range(1, 5001):
    loss, train_op = sess.run([model.loss, model.train_op], feed_dict={
                              handle: train_handle})
    
    if _ % 10 == 0:
        print('After', _, 'iterations:')
        print('Batch Loss:', np.mean(loss))
    
    if _ % 100 == 0:
        acc = 0

        for __ in range(18):
            pred1, target1, pred2, target2 = sess.run([tf.argmax(model.y1, 1), model.yp1,
                                                       tf.argmax(model.y2, 1), model.yp2], feed_dict={handle: dev_handle})
            acc += np.mean(np.logical_and(pred1 == target1, pred2 == target2))

        acc /= float(18)
        print('Dev Accuracy:', acc)
        
        if acc > best_acc:
            best_acc = acc
            saver2.save(sess, 'log/retraining_model/badptr-savepoint', global_step=_)
    
    if _ % 10 == 0:
        print()